In [1]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.metrics import Precision, Recall
from sklearn.utils import class_weight
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
from PIL import Image, ImageEnhance

from tensorflow.keras.utils import Sequence
import numpy as np
import pandas as pd
import os


# Suprimir todos los warnings
import warnings
warnings.filterwarnings('ignore')

In [2]:


# Listar los dispositivos físicos disponibles, en este caso, GPUs
physical_devices = tf.config.list_physical_devices('GPU')

# Si hay alguna GPU disponible
if len(physical_devices) > 0:
    # Permitir que TensorFlow crezca dinámicamente la memoria utilizada en la GPU
    tf.config.experimental.set_memory_growth(physical_devices[0], True)

physical_devices

import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


In [3]:

# Cargar el dataset

#metadata_path = '/Users/luiseduardogarciablanco/Desktop/nueva data cancer/prueba_data_16_20/dataset_16_20_emb.csv'
metadata = pd.read_csv('/Users/luiseduardogarciablanco/Desktop/nueva data cancer/prueba_data_16_20/dataset_16_20_emb.csv')  # Usar la primera columna como índice


# Ruta a las imágenes
image_path = "/Users/luiseduardogarciablanco/Desktop/nueva data cancer/prueba_data_16_20/image"


# Mostrar información básica del dataset
print (metadata.head())
print(metadata['target'].value_counts())

              image_id      sex  age_approx anatom_site_general  target
0  16_ISIC_0000001.jpg  Unknown          -1             Unknown       0
1  16_ISIC_0000002.jpg  Unknown          -1             Unknown       1
2  16_ISIC_0000004.jpg  Unknown          -1             Unknown       1
3  16_ISIC_0000006.jpg  Unknown          -1             Unknown       0
4  16_ISIC_0000007.jpg  Unknown          -1             Unknown       0
target
0    64604
1     6766
Name: count, dtype: int64


In [4]:
metadata = metadata [['image_id', 'target']]
metadata

,image_id,target
0,16_ISIC_0000001.jpg,0
1,16_ISIC_0000002.jpg,1
2,16_ISIC_0000004.jpg,1
3,16_ISIC_0000006.jpg,0
4,16_ISIC_0000007.jpg,0
...,...,...
71365,20_ISIC_9999134.jpg,0
71366,20_ISIC_9999320.jpg,0
71367,20_ISIC_9999515.jpg,0
71368,20_ISIC_9999666.jpg,0


In [5]:
import pandas as pd

# Cargar el dataset
df = metadata

# Dividir el DataFrame en dos partes según el valor del target
df_majority = df[df['target'] == 0]
df_minority = df[df['target'] == 1]

# Reducir el número de muestras de la clase mayoritaria a 6766 para quilibrar las muestras
df_majority_downsampled = df_majority.sample(n=6766, random_state=42)

# Combinar los DataFrames balanceados
df_balanced = pd.concat([df_majority_downsampled, df_minority])

# Opcional: Mezclar el DataFrame balanceado para asegurar aleatoriedad
df_balanced = df_balanced.sample(frac=1, random_state=42).reset_index(drop=True)

# Mostrar la distribución de clases después del balanceo
print(df_balanced['target'].value_counts())

df_balanced.info()

target
0    6766
1    6766
Name: count, dtype: int64
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13532 entries, 0 to 13531
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   image_id  13532 non-null  object
 1   target    13532 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 211.6+ KB


In [6]:
metadata= df_balanced
metadata

,image_id,target
0,20_ISIC_8376677.jpg,0
1,18_ISIC_0024932.jpg,1
2,20_ISIC_3449574.jpg,0
3,20_ISIC_8143299.jpg,0
4,20_ISIC_0189336.jpg,0
...,...,...
13527,18_ISIC_0033910.jpg,0
13528,20_ISIC_8040024.jpg,1
13529,20_ISIC_5667935.jpg,0
13530,20_ISIC_5310630.jpg,0


In [7]:
# Convertir la columna 'target' a string
metadata['target'] = metadata['target'].astype(str)

In [8]:

# Dividir los datos en entrenamiento y validación
train_df, val_df = train_test_split(metadata, test_size=0.2, stratify=metadata['target'], random_state=42)

print (train_df.shape)
print (val_df.shape)

(10825, 2)
(2707, 2)


In [9]:
# Convertir los valores de la columna target a cadenas de texto
train_df['target'] = train_df['target'].astype(str)
val_df['target'] = val_df['target'].astype(str)

In [10]:
# Directorio de las imágenes
image_path = '/Users/luiseduardogarciablanco/Desktop/nueva data cancer/prueba_data_16_20/image'
'''
# Función para ajustar la nitidez
def adjust_sharpness(image):
    # Asegúrate de que la imagen esté en el rango uint8 antes de convertir
    if image.dtype != np.uint8:
        image = (image * 255).astype(np.uint8)  # Ajuste para imagen normalizada
    image = Image.fromarray(image)
    enhancer = ImageEnhance.Sharpness(image)
    image = enhancer.enhance(2.0)  # Aumenta la nitidez
    return np.array(image) / 255.0  # Convierte de vuelta a rango [0, 1]

# Función para ajustar el contraste
def adjust_contrast(image):
    # Asegúrate de que la imagen esté en el rango uint8 antes de convertir
    if image.dtype != np.uint8:
        image = (image * 255).astype(np.uint8)  # Ajuste para imagen normalizada
    image = Image.fromarray(image)
    enhancer = ImageEnhance.Contrast(image)
    image = enhancer.enhance(1.5)  # Aumenta el contraste
    return np.array(image) / 255.0  # Convierte de vuelta a rango [0, 1]

# Función de preprocesamiento que combina ambas
def preprocess_function(image):
    image = adjust_sharpness(image)
    image = adjust_contrast(image)
    return image

# Crear generadores de datos con augmentación
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    validation_split=0.2,  # División para validación
    preprocessing_function=preprocess_function  # Añade la función personalizada
)

'''
# Crear generadores de datos con augmentación
train_datagen = ImageDataGenerator(
    #rescale=1./255,
    #rotation_range=20,
    #width_shift_range=0.2,
    #height_shift_range=0.2,
    #horizontal_flip=True,'''
    validation_split=0.2  # División para validación
)
#generador de entrenamiento
train_gen = train_datagen.flow_from_dataframe(
    dataframe=metadata,
    directory=image_path,
    x_col='image_id',
    y_col='target',
    target_size=(256, 256),
    batch_size=32,
    class_mode='binary',
    subset='training',
    shuffle=True
)
#generador de validacion
val_gen = train_datagen.flow_from_dataframe(
    dataframe=metadata,
    directory=image_path,
    x_col='image_id',
    y_col='target',
    target_size=(256, 256),
    batch_size=32,
    class_mode='binary',
    subset='validation',
    shuffle=False
)

Found 10826 validated image filenames belonging to 2 classes.
Found 2706 validated image filenames belonging to 2 classes.


In [23]:
from tensorflow.keras.regularizers import l1_l2
# Cargar el modelo preentrenado ResNet50 (sin la parte superior)
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(256, 256, 3))

# Congelar las capas del modelo base
for layer in base_model.layers:
    layer.trainable = False

# Añadir capas adicionales
model = models.Sequential([
    base_model,
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(1, activation='sigmoid',kernel_regularizer=l1_l2(l1=1e-5, l2=1e-4))
    #layers.Dense(1, activation='sigmoid')
])

# Compilar el modelo
model.compile(optimizer=Adam(learning_rate=0.00001), loss='binary_crossentropy', metrics=['accuracy', Precision(), Recall(), tf.keras.metrics.AUC(name='auc')])
#model.compile(optimizer=Adam(learning_rate=1e-4), loss='binary_crossentropy', metrics=['accuracy', Precision(), Recall(), tf.keras.metrics.AUC(name='auc')])

# Entrenamiento inicial (solo las capas superiores)
history = model.fit(
    train_gen,
    epochs=5,
    validation_data=val_gen
)

# Descongelar algunas capas del modelo base para fine-tuning
#for layer in base_model.layers[-10:]:# probaremos a dscongelar 20, 40 y 60 capas para comprobar la eficacia del modelo
for layer in base_model.layers[-40:]:
    layer.trainable = True

# Compilar de nuevo con una tasa de aprendizaje más baja
model.compile(optimizer=Adam(learning_rate=1e-5), loss='binary_crossentropy', metrics=['accuracy', Precision(), Recall(), tf.keras.metrics.AUC(name='auc')])

# Calcular los pesos de clase
class_weights = class_weight.compute_class_weight('balanced', classes=np.unique(metadata['target']), y=metadata['target'])
class_weights = dict(enumerate(class_weights))

# Entrenamiento con Fine-Tuning
history_fine = model.fit(
    train_gen,
    epochs=10,
    validation_data=val_gen,
    #class_weight=class_weights, # debe desactivarse si los datos están balanceados
    callbacks=[tf.keras.callbacks.EarlyStopping(patience=3, restore_best_weights=True)]
)

# Evaluar el modelo para obtener el AUC-ROC
evaluation = model.evaluate(val_gen)
print(f"Test Loss: {evaluation[0]}")
print(f"Test Accuracy: {evaluation[1]}")
print(f"Test Precision: {evaluation[2]}")
print(f"Test Recall: {evaluation[3]}")


Epoch 1/5


2024-08-30 16:45:51.481405: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


339/339 [==============================] - ETA: 0s - loss: 0.8119 - accuracy: 0.7283 - precision_2: 0.7296 - recall_2: 0.7289 - auc: 0.8015

2024-08-30 16:46:52.353526: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


339/339 [==============================] - 77s 222ms/step - loss: 0.8119 - accuracy: 0.7283 - precision_2: 0.7296 - recall_2: 0.7289 - auc: 0.8015 - val_loss: 0.5599 - val_accuracy: 0.7772 - val_precision_2: 0.7261 - val_recall_2: 0.8792 - val_auc: 0.8662
Epoch 2/5
339/339 [==============================] - 73s 215ms/step - loss: 0.6080 - accuracy: 0.7753 - precision_2: 0.7756 - recall_2: 0.7769 - auc: 0.8552 - val_loss: 0.4710 - val_accuracy: 0.7997 - val_precision_2: 0.7772 - val_recall_2: 0.8320 - val_auc: 0.8835
Epoch 3/5
339/339 [==============================] - 73s 215ms/step - loss: 0.5183 - accuracy: 0.8021 - precision_2: 0.8008 - recall_2: 0.8064 - auc: 0.8824 - val_loss: 0.4874 - val_accuracy: 0.8067 - val_precision_2: 0.7914 - val_recall_2: 0.8252 - val_auc: 0.8863
Epoch 4/5
339/339 [==============================] - 73s 215ms/step - loss: 0.4608 - accuracy: 0.8195 - precision_2: 0.8166 - recall_2: 0.8259 - auc: 0.9010 - val_loss: 0.4971 - val_accuracy: 0.8156 - val_precisi

Epoch 1/10


2024-08-30 16:52:00.012223: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


339/339 [==============================] - ETA: 0s - loss: 0.6443 - accuracy: 0.8002 - precision_3: 0.8010 - recall_3: 0.8008 - auc: 0.8801

2024-08-30 16:53:25.300214: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


339/339 [==============================] - 103s 294ms/step - loss: 0.6443 - accuracy: 0.8002 - precision_3: 0.8010 - recall_3: 0.8008 - auc: 0.8801 - val_loss: 0.6820 - val_accuracy: 0.7967 - val_precision_3: 0.7371 - val_recall_3: 0.9130 - val_auc: 0.8877
Epoch 2/10
339/339 [==============================] - 97s 285ms/step - loss: 0.4125 - accuracy: 0.8618 - precision_3: 0.8623 - recall_3: 0.8623 - auc: 0.9335 - val_loss: 0.6059 - val_accuracy: 0.8234 - val_precision_3: 0.7833 - val_recall_3: 0.8867 - val_auc: 0.9012
Epoch 3/10
339/339 [==============================] - 96s 284ms/step - loss: 0.2779 - accuracy: 0.8996 - precision_3: 0.8992 - recall_3: 0.9010 - auc: 0.9631 - val_loss: 0.7120 - val_accuracy: 0.8204 - val_precision_3: 0.7603 - val_recall_3: 0.9280 - val_auc: 0.9061
Epoch 4/10
339/339 [==============================] - 97s 285ms/step - loss: 0.1904 - accuracy: 0.9315 - precision_3: 0.9316 - recall_3: 0.9319 - auc: 0.9803 - val_loss: 0.5796 - val_accuracy: 0.8437 - val_pre

In [27]:
# Evaluar el modelo
loss, accuracy, precision, recall, auc= model.evaluate(val_gen)

85/85 [==============================] - 14s 166ms/step - loss: 0.5796 - accuracy: 0.8437 - precision_3: 0.8227 - recall_3: 0.8702 - auc: 0.9108


In [32]:
# Predecir las probabilidades
y_pred_probs = model.predict(val_gen)

# Ajustar el umbral de decisión
threshold = 0.5 # Ajustar según sea necesario
y_pred_adjusted = (y_pred_probs > threshold).astype(int)

# Calcular y mostrar métricas
from sklearn.metrics import classification_report

y_true = val_gen.classes
print(classification_report(y_true, y_pred_adjusted, target_names=['Class 0', 'Class 1']))

85/85 [==============================] - 13s 160ms/step
              precision    recall  f1-score   support

     Class 0       0.87      0.82      0.84      1373
     Class 1       0.82      0.87      0.85      1333

    accuracy                           0.84      2706
   macro avg       0.84      0.84      0.84      2706
weighted avg       0.84      0.84      0.84      2706



descongelando 10 capas conseguimos el siguiente resultado:

85/85 [==============================] - 19s 214ms/step
              precision    recall  f1-score   support

     Class 0       0.84      0.79      0.82      1373
     Class 1       0.80      0.85      0.82      1333

    accuracy                           0.82      2706
   macro avg       0.82      0.82      0.82      2706
weighted avg       0.82      0.82      0.82      2706



descongelando 20 capas conseguimos el sigiuente resultado:

85/85 [==============================] - 16s 177ms/step
              precision    recall  f1-score   support

     Class 0       0.84      0.82      0.83      1373
     Class 1       0.82      0.84      0.83      1333

    accuracy                           0.83      2706
   macro avg       0.83      0.83      0.83      2706
weighted avg       0.83      0.83      0.83      2706



descongelando 40 dio este resultado:
85/85 [==============================] - 16s 179ms/step
              precision    recall  f1-score   support

     Class 0       0.87      0.82      0.85      1373
     Class 1       0.82      0.88      0.85      1333

    accuracy                           0.85      2706
   macro avg       0.85      0.85      0.85      2706
weighted avg       0.85      0.85      0.85      2706



descongelando 60 dio este resultado:
85/85 [==============================] - 15s 167ms/step
              precision    recall  f1-score   support

     Class 0       0.85      0.83      0.84      1373
     Class 1       0.83      0.85      0.84      1333

    accuracy                           0.84      2706
   macro avg       0.84      0.84      0.84      2706
weighted avg       0.84      0.84      0.84      2706



In [33]:
# Guardar el modelo entrenado
model.save('resnet50_fine_tuned_model_40.h5')